### Calculate RR using PM2.5

This code calculates the RR from Peng et al., 2021 and GEMM parameter estimates from Burnett et al., 2018. 

RR is calculated for adults >25 using 5-year interval age groups

In [2]:
import geopandas as gpd
import xarray as xr
from cartopy import crs as ccrs
import seaborn as sns; sns.set_theme()
import os
import fiona
import country_converter as coco
import dask
import dask.array as da
import netCDF4 as nc
import regionmask
from matplotlib import cm
import numpy as np
from matplotlib import pyplot as plt
import country_converter as coco
import pyogrio
#pyogrio.set_gdal_config_options({"SHAPE_RESTORE_SHX": "YES"})
import pandas as pd
from cartopy.util import add_cyclic_point
import nc_time_axis
import glob
import cdo
import pandas as pd
import cartopy.feature as cfeature

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
num_ensembles = 10
var_list = 'PM25'.split()
ensembles = {var: [] for var in var_list}
# Loop over each ensemble member
for i in range(1, num_ensembles + 1):
    ensemble_id = f'{i:03d}'  # Format ensemble number as 001, 002, ..., 010
    print(f'Loading ensemble member {ensemble_id}')
    
    # Loop over each variable
    for var in var_list:
        # Construct the file path
        #if var == 'TS':
            # Special case for TS (daily data)
        #    file_path = f'/glade/campaign/cesm/collections/ARISE-SAI-1.5/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.{ensemble_id}/atm/proc/tseries/day_1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.{ensemble_id}.cam.h1.TS.*.nc'
        file_path = f'/glade/derecho/scratch/cindywang625/regrid_PM25/new/{ensemble_id}.PM25_new.nc'
        #file_path = f'/glade/campaign/cesm/collections/ARISE-SAI-1.5/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.{ensemble_id}/atm/proc/tseries/month_1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.{ensemble_id}.cam.h0.{var}.*.nc'
        
        # Load the dataset and extract the variable
        ds = xr.open_mfdataset(file_path, parallel=True, combine='nested', concat_dim='time')
        ensembles[var].append(ds[var])

Loading ensemble member 001
Loading ensemble member 002
Loading ensemble member 003
Loading ensemble member 004
Loading ensemble member 005
Loading ensemble member 006
Loading ensemble member 007
Loading ensemble member 008
Loading ensemble member 009
Loading ensemble member 010


In [5]:
average_ensemble = {}
for var in var_list:
    # Concatenate all ensemble members along a new dimension
    combined = xr.concat(ensembles[var], dim='ensemble')
    # Calculate the mean over the ensemble dimension
    average_ensemble[var] = combined.mean(dim='ensemble')

In [6]:
#base_ssp245 = {}
base_ssp245 = {var: [] for var in var_list}
for i in range(1, num_ensembles + 1):
    ensemble_id = f'{i:03d}'  # Format ensemble number as 001, 002, ..., 010
    print(f'Loading ensemble member {ensemble_id}')
    
    # Loop over each variable
    for var in var_list:
        # Construct the file path
        #if var == 'TS':
            # Special case for TS (daily data)
        #    file_path = f'/glade/campaign/cesm/collections/ARISE-SAI-1.5/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.{ensemble_id}/atm/proc/tseries/day_1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.{ensemble_id}.cam.h1.TS.*.nc'
        file_path = f'/glade/derecho/scratch/cindywang625/regrid_PM25/ssp245/{ensemble_id}.PM25_new.nc'
        #file_path = f'/glade/campaign/cesm/collections/ARISE-SAI-1.5/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.{ensemble_id}/atm/proc/tseries/month_1/b.e21.BW.f09_g17.SSP245-TSMLT-GAUSS-DEFAULT.{ensemble_id}.cam.h0.{var}.*.nc'
        
        # Load the dataset and extract the variable
        ds = xr.open_mfdataset(file_path, parallel=True, combine='nested', concat_dim='time')
        base_ssp245[var].append(ds[var])

Loading ensemble member 001
Loading ensemble member 002
Loading ensemble member 003
Loading ensemble member 004
Loading ensemble member 005
Loading ensemble member 006
Loading ensemble member 007
Loading ensemble member 008
Loading ensemble member 009
Loading ensemble member 010


In [7]:
average_ensemble_ssp245 = {}
for var in var_list:
    # Concatenate all ensemble members along a new dimension
    combined = xr.concat(base_ssp245[var], dim='ensemble')
    # Calculate the mean over the ensemble dimension
    average_ensemble_ssp245[var] = combined.mean(dim='ensemble')

In [8]:
def global_mean_xarray(ds_XXLL):
    """ 
    Compute the global mean value of the data.
    The data has to have the lat and lon in its dimensions.
    Should not include NaN in Inputs.
    
    Parameters
    ----------
    ds_XXLL   : xarray with lat and lon. ds_XXLL.lat will be 
                used for area weight.

    Returns
    ----------
    tmp_XX    : xarray without lat and lon.
    
    """
    lat = ds_XXLL.coords['lat']        # readin lat
    # global mean
    # compute cos(lat) as a weight function
    weight_lat = np.cos(np.deg2rad(lat))/np.mean(np.cos(np.deg2rad(lat)))
    tmp_XXL = ds_XXLL.mean(dim=['lon'])*weight_lat
    tmp_XX  = tmp_XXL.mean(dim=['lat'])
    return tmp_XX

def weighted_temporal_mean_l(ds, var=None):
    """
    weight by days in each month
    """
    #ds = xr.decode_cf(ds)
    # Determine the month length
    month_length = ds.time.dt.days_in_month

    # Calculate the weights
    wgts = month_length.groupby("time.year") / month_length.groupby("time.year").sum()

    # Make sure the weights in each year add up to 1
    np.testing.assert_allclose(wgts.groupby("time.year").sum(xr.ALL_DIMS), 1.0)

    # Subset our dataset for our variable
    obs = ds if var is None else ds[var]

    # Setup our masking for nan values
    cond = obs.isnull()
    ones = xr.where(cond, 0.0, 1.0)

    # Calculate the numerator
    obs_sum = (obs * wgts).resample(time="AS").sum(dim="time")

    # Calculate the denominator
    ones_out = (ones * wgts).resample(time="AS").sum(dim="time")

    # Return the weighted average
    return obs_sum / ones_out

## Calculate relative risk (RR) for the different age groups

RR of NCDS+LRIs associated with PM2.5 exposure is calculated for adults >=25 years old using 5-year interval age groups

According to GEMM, RR of NCDS+LRI is calculated as follows:

\begin{equation}
RR(C)=e^{\frac{\theta\times log(\frac{C}{\alpha + 1})}{1+e\frac{C-\mu}{V}}}, C = max(0, C-2.4)
\end{equation}

C is the long-term ambient PM2.5 concentration; $\theta$, $\mu$ and $v$ are parameters that determine the shape of RR in GEMM. The parameters are constant across all age groups for NCD+LRI, while $\theta$ varies by age group.

In [9]:
#GEMM parameters from Brunett et al.,
mu = 15.5
alpha = 1.6
v = 36.8

In [10]:
# The age range are as follows
# >25 - 0.1430
# Age ranges and their corresponding theta values
age_theta = {
    '25to30': 0.1585,
    '30to35': 0.1577,
    '35to40': 0.1570,
    '40to45': 0.1558,
    '45to50': 0.1532,
    '50to55': 0.1499,
    '55to60': 0.1462,
    '60to65': 0.1421,
    '65to70': 0.1374,
    '70to75': 0.1319,
    '75to80': 0.1253,
    '80to85': 0.1141
}

## Reiterate for each ensemble member and ensemble average

In [101]:
# DOUBLE CHECK that time = 35 (2035 to 2069; ens 7 and 8 are run up to 2071)
i = 9 #<<<<--- change this for each ensemble i=0 to 9

#do it for 2020 for now
C_base = weighted_temporal_mean_l(base_ssp245['PM25'][i])[20:-1]*1e9
#C_base = weighted_temporal_mean_l(average_ensemble_ssp245['PM25'])[20:-1]*1e9

#C = np.maximum(0, C - 2.4)
C_em = weighted_temporal_mean_l(ensembles['PM25'][i])[:35]*1e9
#C_em = weighted_temporal_mean_l(average_ensemble['PM25'])[:-2]*1e9


In [89]:
#Precompute common terms
log_term_base = np.log10(C_base/(alpha+1))
exp_term_base = np.exp(-(C_base-mu)/v)

#Precompute common terms
log_term_em = np.log10(C_em/(alpha+1))
exp_term_em = np.exp(-(C_em-mu)/v)

In [90]:
# Initialize a blank 3D array for RR
num_age_ranges = len(age_theta)
RR = np.zeros((num_age_ranges, C_base.shape[1], C_base.shape[2]))
RR_val_base = np.zeros((num_age_ranges, C_base.shape[0], C_base.shape[1],C_base.shape[2]))
RR_val_em = np.zeros((num_age_ranges, C_base.shape[0], C_base.shape[1],C_base.shape[2]))



In [91]:
calcs = xr.Dataset()
calcs.coords['age'] = (('age'), list(age_theta))
calcs.coords['time'] = (('time'), C_base['time'].data)

calcs.coords['lat']  = (('lat'),C_base.coords['lat'].values)
calcs.coords['lon']  = (('lon'),C_base.coords['lon'].values)
#calcs['RR']      = (('age','lat','lon'),RR)

In [92]:
for j in range(C_base['time'].shape[0]):
    logged = log_term_base[j]
    expped = exp_term_base[j]
    for i in range(12):
        theta_values = np.array(list(age_theta.values()))[i]
        RR[i] = np.exp((theta_values*logged)/(1+expped))
        #RR[:,C[j] < 2.4] = 1
        #RR[RR<1] = 1
        #RR[RR==0] = np.nan
        RR_val_base[:,j] = RR

In [93]:
for j in range(C_em['time'].shape[0]):
    logged = log_term_em[j]
    expped = exp_term_em[j]
    for i in range(12):
        theta_values = np.array(list(age_theta.values()))[i]
        RR[i] = np.exp((theta_values*logged)/(1+expped))
        #RR[:,C[j] < 2.4] = 1
        #RR[RR<1] = 1
        #RR[RR==0] = np.nan
        RR_val_em[:,j] = RR

In [94]:
RR_val_base[RR_val_base == 0] = np.nan
RR_val_base[RR_val_base<1] =1

RR_val_em[RR_val_em == 0] = np.nan
RR_val_em[RR_val_em<1] =1

RR is calculated separatedly for SSP2-4.5 and the ARISE simulations. But for the calculations, they are subtracted:

\begin{equation}
 \Delta RR = \frac{1}{RR_{base}} - \frac{1}{RR_{perturb}}
\end{equation}

In [95]:
calcs['RR_em']      = (('age','time','lat','lon'),1/RR_val_em)
calcs['RR_base']      = (('age','time','lat','lon'),1/RR_val_base)
calcs['RR_diff']      = (('age','time','lat','lon'),(1/RR_val_base)-(1/RR_val_em))

In [96]:
calcs.to_netcdf('/glade/derecho/scratch/cindywang625/RR/010.nc')
